In [ ]:
!pip install pandas_profiling

In [ ]:
import pandas as pd
import numpy as np
import re 
#import geopandas as gpd
import pandas as pd
import json
import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_theme(style="white")
import warnings
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
from pandas_profiling import ProfileReport
import random
from sklearn import metrics 
from scipy.spatial.distance import cdist

In [ ]:
budgets_fbi = pd.read_csv('CleanData/Merged_FBI_Budgets_Normalized.csv')
budgets_fbi.head()

### Load the Income Data

In [ ]:
income_data= pd.read_csv('CleanData/DPI_Clean.csv')
povertydf = pd.read_csv('CleanData/Demographics.csv')
racedf = pd.read_csv('CleanData/Race.csv')
racedf = racedf[racedf['geographic_area_name'].notnull()]


income_data.loc[income_data.loc[income_data['city'].str.contains('Fort Smith, AR')].index.values,'city'] = 'Ft. Smith, AR'
income_data.loc[income_data.loc[income_data['city'].str.contains('Boise City, ID')].index.values,'city'] = 'Boise, ID'
income_data.loc[income_data.loc[income_data['city'].str.contains('Louisville/Jefferson County, KY')].index.values,'city'] = 'Louisville, KY'

povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Fort Smith, AR')].index.values,'city_name'] = 'Ft. Smith, AR'
povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Boise City, ID')].index.values,'city_name'] = 'Boise, ID'
povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Louisville/Jefferson County, KY')].index.values,'city_name'] = 'Louisville, KY'

racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Fort Smith, AR')].index.values,'geographic_area_name'] = 'Ft. Smith, AR'
racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Boise City, ID')].index.values,'geographic_area_name'] = 'Boise, ID'
racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Louisville/Jefferson County, KY')].index.values,'geographic_area_name'] = 'Louisville, KY'

In [ ]:
merged_df = budgets_fbi.merge(income_data,left_on=['city_merge_name','year'],right_on = ['city','Year'],how='inner')
merged_df.city.unique().shape

### Lost Cities

In [ ]:
for i in budgets_fbi.city_merge_name.unique():
    if i not in merged_df.city.unique():
        print(i)

In [ ]:
merged_df2 = merged_df.merge(povertydf,left_on=['city_merge_name','year'],right_on = ['city_name','Year'],how='inner')
merged_df2.city.unique().shape

In [ ]:
for i in merged_df.city_merge_name.unique():
    if i not in merged_df2.city.unique():
        print(i)

In [ ]:
merged_df3 = merged_df2.merge(racedf,left_on=['city_merge_name','year'],right_on = ['geographic_area_name','Year'],how='inner')
merged_df3 = merged_df3.T.drop_duplicates().T


merged_df3['percent_race_one_race_asian'] =merged_df3['percent_race_one_race_asian'].str.replace('N','0.0')
merged_df3['percent_race_one_race_black_or_african_american'] = merged_df3['percent_race_one_race_black_or_african_american'].str.replace('N','0.0')
merged_df3['percent_race_one_race_native_hawaiian_and_other_pacific_islander'] = merged_df3['percent_race_one_race_native_hawaiian_and_other_pacific_islander'].str.replace('N','0.0')
merged_df3['percent_race_one_race_white'] = merged_df3['percent_race_one_race_white'].str.replace('N','0.0')
merged_df3= merged_df3.drop(columns=['city_population'])
merged_df3.head()

In [ ]:
norm_column = "population_persons"


cols_to_normalize = ["personal_income_thousands_of_dollars",
"total_estimate_educational_attainment_population_25_years_and_over",
"total_estimate_age_18_to_64_years",
"total_estimate_employment_status_civilian_labor_force_16_years_and_over",
"total_estimate_population_for_whom_poverty_status_is_determined",
"total_estimate_age_65_years_and_over",
"total_estimate_sex_female",
"total_estimate_work_experience_population_16_years_and_over",
"total_estimate_age_under_18_years",
"total_estimate_sex_male",
"total_estimate_age_65_years_and_over"]


for i in cols_to_normalize:
    merged_df3[i] = merged_df3[i]/merged_df3["population_persons"]

In [ ]:
merged_df3

In [ ]:
dfoi = merged_df3[merged_df3.year==2017][merged_df3.columns[:9]].head(10)
dfoi 
profile = ProfileReport(dfoi)

In [ ]:
def convert_to_numeric(df,col):
    try:
        df[col] = df[col].apply(float)
    except:
        print(col)
    
def convert_name_to_numeric(row):
    return name_dict[row]

def numeric_to_name(row):
    return label_dict[row]
    
    

### Create The Df for clustering

In [ ]:
test= merged_df3.copy().dropna()
years= test.year
test = test.drop(columns='population')
test = test.drop(columns="population_persons")

for i in test.columns:
    convert_to_numeric(test,i)
test = test.drop(columns='year')
test

## Create mappings between names and numerics so we can do clustering

In [ ]:
name_dict = {}
label_dict = {}
for i,item in enumerate(merged_df3['city_merge_name'].unique()):
    name_dict[item] = i
    label_dict[i]=item
name_dict

In [ ]:
test['city_merge_name'] = test['city_merge_name'].apply(convert_name_to_numeric)

distortions = [] 
inertias = []
mapping1 = {}
mapping2 = {} 
K = range(1,15) 

X = test.values[:,:]
X = np.nan_to_num(X)
#Normalize data
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset
    
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(init="k-means++", n_clusters=k, n_init=50)
    kmeanModel.fit(cluster_dataset)     
      
    distortions.append(sum(np.min(cdist(cluster_dataset, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / cluster_dataset.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(cluster_dataset, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / cluster_dataset.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 
    
test['city_merge_name'] = test['city_merge_name'].apply(numeric_to_name)   
for key,val in mapping1.items(): 
    print(str(key)+' : '+str(val)) 
        
# Make Plot 1

plt.figure(figsize=(15,8))
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 
    
for key,val in mapping2.items(): 
    print(str(key)+' : '+str(val)) 
    
# Make Plot 2

plt.figure(figsize=(15,8))
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show()

In [ ]:
num_clusters = 6
test['city_merge_name'] = test['city_merge_name'].apply(convert_name_to_numeric)

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=50,algorithm = "elkan")
k_means.fit(cluster_dataset)
labels = k_means.labels_

test['city_merge_name'] = test['city_merge_name'].apply(numeric_to_name)
test["Labels"] = labels
test['Year'] = years

In [ ]:
modes = test.groupby(['city_merge_name','Labels']).count().reset_index().sort_values(by='violent_crime')
top_labels = modes[['city_merge_name','Labels','violent_crime']].groupby('city_merge_name').head(1)[['city_merge_name','Labels']]
labels = top_labels.sort_values('Labels')
summary = labels.groupby('Labels').head(10).rename(columns={"city_merge_name":"City","Labels":"Group"}).reset_index().drop(columns="index")
summary

In [ ]:
test[test.Year==2017].groupby("Labels").count()

In [ ]:
def apply_labels(row):
    item = labels['Labels'][labels.city_merge_name==row].values[0]
    return item

In [ ]:
test['Group']=test['city_merge_name'].apply(apply_labels)

In [ ]:
res= test.groupby('Group').median().reset_index()
res.sort_values(by='violent_crime',ascending=False)

In [ ]:
crime = res.copy()
crime[["violent_crime","property_crime"]] = crime[["violent_crime","property_crime"]]*(10**5)
plt.figure(figsize=(15,8))
ax = sns.barplot(data=crime,x='Group',y='violent_crime',palette="crest")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Rates of Violent Crime', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Rate of Violent Crime (per 100,000)',color='#5B5B5B')

In [ ]:
crime = res.copy()
crime[["violent_crime","property_crime"]] = crime[["violent_crime","property_crime"]]*(10**5)
plt.figure(figsize=(15,8))
ax = sns.barplot(data=crime,x='Group',y='property_crime',palette="crest")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Rates of Violent Crime', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Rate of Violent Crime (per 100,000)',color='#5B5B5B')

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.barplot(data=res,x='Group',y='police',palette="crest")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Police Spending per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Police Spending (per capita $)',color='#5B5B5B')

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.barplot(data=res,x='Group',y='education_services',palette="crest")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Education Services Spending per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Education Services Spending (per capita $)',color='#5B5B5B')

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.barplot(data=res,x='Group',y='social_services',palette="crest")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Social Services Spending per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Social Services Spending (per capita $)',color='#5B5B5B')

### Let's Look at Trends

In [ ]:
res= test.groupby(['Year','Group']).median().reset_index().sort_values(by=['Year','violent_crime'],ascending=[True,False])
res['Year'] = pd.to_datetime(res['Year'],format="%Y")

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.lineplot(data=res[res.Group==4],x='Year',y='police',palette="Spectral",hue="Group")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Police Spending per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Police Spending (per capita $)',color='#5B5B5B')

In [ ]:
plt.figure(figsize=(25,10))
ax = sns.lineplot(data=res[res.Group.isin([1,2,3,5])],x='Year',y='police',palette="deep",hue="Group")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Police Spending per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Police Spending (per capita $)',color='#5B5B5B')
_= plt.legend(loc = 'upper left',frameon=False)

In [ ]:
plt.figure(figsize=(25,10))
ax = sns.lineplot(data=res[res.Group.isin([1,2,3,5])],x='Year',y='education_services',palette="deep",hue="Group")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Education Services per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Education Services (per capita $)',color='#5B5B5B')
_= plt.legend(loc = 'upper left',frameon=False)

In [ ]:
plt.figure(figsize=(25,10))
ax = sns.lineplot(data=res[res.Group.isin([1,2,3,5])],x='Year',y='social_services',palette="deep",hue="Group")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Social Services per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Social Services (per capita $)',color='#5B5B5B')
_= plt.legend(loc = 'upper left',frameon=False)

In [ ]:
pct_changes = res[res.Group==2].copy()
pct_changes['police'] = pct_changes['police'].pct_change().fillna(0.0)
pct_changes['social_services'] = pct_changes['social_services'].pct_change().fillna(0.0)
pct_changes['education_services'] = pct_changes['education_services'].pct_change().fillna(0.0)
pct_changes['violent_crime'] = pct_changes['violent_crime'].pct_change().fillna(0.0)
pct_changes

plt.figure(figsize=(15,10))
ax = sns.lineplot(data=pct_changes,x='Year',y='social_services',palette="deep")
sns.lineplot(data=pct_changes,ax=ax,x='Year',y='police',color='r',palette="deep")
sns.lineplot(data=pct_changes,ax=ax,x='Year',y='education_services',color='g',palette="deep")
sns.lineplot(data=pct_changes,ax=ax,x='Year',y='violent_crime',color='orange',palette="deep")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B",
               grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Social Services per Capita', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
_ = ax.set_ylabel('Social Services (per capita $)',color='#5B5B5B')
_= plt.legend(labels = ['social services','police','education_services','violent_crime'],loc = 'upper left',frameon=False)

### Highest Crime Cities

In [ ]:
bygroup = res.sort_values(by='violent_crime',ascending=False)
highest_crime = int(bygroup.reset_index().loc[0]['Group'])
print("Highest Crime Group:",highest_crime)
corrs = test[test.Labels==highest_crime].corr()
sns.heatmap(corrs[abs(corrs)>0.2])

In [ ]:
ProfileReport(test[test.Group==highest_crime][list(test.columns[:5])+list(test.columns[17:18])+list(test.columns[23:34])])

### Lowest Crime Cities

In [ ]:
lowest_crime = int(bygroup.reset_index().loc[bygroup.shape[0]-2]['Group'])
lowest_crime

In [ ]:
test[test.Group==lowest_crime][list(test.columns[:5])+list(test.columns[17:18])+list(test.columns[23:34])+['total_estimate_mean_income_deficit_for_unrelated_individuals_(dollars)']].corr()

### Strongest Positive Correlations

In [ ]:
violent_crime = test.groupby('Labels').median().corr()['violent_crime']
strongest_positive = list(violent_crime[violent_crime>0.50].index)
strongest_positive

### Strongest Negative Correlations

In [ ]:
violent_crime = test.groupby('Labels').median().corr()['violent_crime']
strongest_negative = list(violent_crime[violent_crime<-0.50].index)
strongest_negative

In [ ]:
test

In [ ]:
relevant_columns = test[['city_merge_name','Year']+strongest_positive+strongest_negative+['Labels']]

In [ ]:
ProfileReport(relevant_columns)

In [ ]:
ProfileReport(test[test.Group==lowest_crime][list(test.columns[:5])+list(test.columns[17:18])+list(test.columns[23:34])])

In [ ]:
res= test.groupby(['Group','Year']).mean().reset_index().sort_values(by=["Year","Group"])
res.head()

In [ ]:
plt.figure(figsize=(15,8))


ax = sns.lineplot(data = res, x='Year',y='violent_crime',hue="Group",palette="deep")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B")
               #grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Violent Crime Over Time', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
ax.set_ylabel('Violent Crime Rate (Per Capita)',color='#5B5B5B')

In [ ]:
test.columns

In [ ]:
plt.figure(figsize=(15,8))


ax = sns.lineplot(data = res, x='Year',y='violent_crime',hue="Group",palette="deep")
ax.spines['bottom'].set_color('#dddddd')
ax.spines['left'].set_color('#dddddd')
sns.despine(left=False, bottom=False, right=True)
ax.tick_params(direction='out', length=6, width=2, colors="#5B5B5B")
               #grid_color='r', grid_alpha=0.5)
_ = ax.set_title('Violent Crime Over Time', color='#5B5B5B',loc='left',pad=25.0,fontweight="bold")
ax.set_ylabel('Violent Crime Rate (Per Capita)',color='#5B5B5B')

### Save Data

In [ ]:
merged_df3.to_csv('CleanData/CompleteMerged.csv',index=False)